In [68]:
# импортируем библиотеки
import pulp
from pulp import *
import random as r

In [69]:
# начальные значения
t0 = 5 # временные границы от 0 до t0
N = 5 # число проектов
B0 = [200, 200, 200, 300, 300] # бюджет по годам
b0 = 1200 # суммарный бюджет

In [70]:
# создаём имена переменных
c_name = ['do_or_not_' + str(i+1) for i in range(N)] # переменные исключения проекта из портфеля
x_name = ['proj_' + str(i+1) for i in range(N)] # переменные времени начала проекта
x_i = [LpVariable(x_name[i], cat = 'Integer') for i in range(N)] # массив переменных библиотеки pulp
c_i = [LpVariable(c_name[i], lowBound = 0, upBound = 1, cat='Binary') for i in range(N)] # массив переменных библиотеки pulp

In [71]:
print(x_i)
print(c_i)

[proj_1, proj_2, proj_3, proj_4, proj_5]
[do_or_not_1, do_or_not_2, do_or_not_3, do_or_not_4, do_or_not_5]


In [72]:
# задаём рандомные значения для проектов
table = []
for i in range(N): # для каждого проекта задаём:
    mini = [0, 0, 0, 0, 0, 0, 0, 0]
    mini[0] = r.randint(1,6) # длительность
    mini[1] = r.randint(0, t0) # начать не позже чем
    mini[2] = r.randint(mini[1], t0) # закончить не позже чем
    mini[3] = r.randint(0, 5) # лаг
    mini[4] = r.randint(100, 200) # годовой расход
    mini[5] = r.randint(200, 300) # годовой доход
    mini[6] = r.randint(0, 1) # обязательный или необязательный проект
    # задаём будет ли проект зависеть от других проектов (р=25%)
    a = r.randint(0,4)
    if (a == 0):
        mini[7] = r.randint(1, N)
    else:
        mini[7] = -1
    table.append(mini)
print(table)

[[6, 5, 5, 5, 110, 293, 0, -1], [5, 4, 4, 5, 107, 268, 1, -1], [5, 3, 4, 1, 132, 253, 1, -1], [6, 2, 5, 3, 195, 241, 0, -1], [5, 0, 2, 4, 102, 300, 0, -1]]


In [145]:
# задаём начальные значения из таблицы
s1 = [-10, 5, 5,0,5]
s2 = [200, 150, 100, 200, 200]
time = [1,2,4,3,2]
for i in range(N):
    table[i][0] = time[i]
    table[i][1] = t0
    table[i][2] = t0
    table[i][3] = 0
    table[i][6] = 0
    table[i][4] = s2[i]
    table[i][5] = s1[i]
    table[i][7] = -1
table[4][3] = 1
table[0][6] = 1
table[1][7] = 1
table[3][7] = 2
table[4][7] = 4

for i in range(N):
    print(table[i])

[1, 5, 5, 0, 200, -10, 1, -1]
[2, 5, 5, 0, 150, 5, 0, 1]
[4, 5, 5, 0, 100, 5, 0, -1]
[3, 5, 5, 0, 200, 0, 0, 2]
[2, 5, 5, 1, 200, 5, 0, 4]


In [146]:
# создаём задачу и говорим, что будем максимизировать целевую функцию
portfolio = LpProblem("GazProm", LpMaximize)

In [147]:
# задаём целевую функцию и добавляем её в задачу
# аффинная сумма между ci и (доходы - расходы) * ti
function = LpAffineExpression([(c_i[i], (table[i][5]) * table[i][0]) for i in range(N)])
portfolio += function, "max"

In [148]:
# добавляем условия в задачу
# для каждого из проектов:
for i in range(N):
    if (table[i][7] != -1): # если проект зависит от другого проекта:
        # xi >= xj + tj + tau_i + (1-ci)*2*t0
        portfolio += x_i[i] - x_i[table[i][7] - 1] + c_i[i] * t0 >= table[table[i][7] - 1][0] + table[i][3] + t0, "prev : " + str(i+1)
        # ci <= cj
        portfolio += c_i[i] - c_i[table[i][7] - 1] <= 0, "prev c_i : " + str(i+1)
    else:
        # xi >= (1-ci)*2*t0
        portfolio += x_i[i] + c_i[i] * t0 >= t0, "no prev : " + str(i+1)
        # ci <= 1
        portfolio += c_i[i] <= 1, "no prev c_i : " + str(i+1)
    # xi <= t_st_i + (1-ci)*(5*t0-t_st_i)
    portfolio += x_i[i] + c_i[i] * (10 * t0 - table[i][1]) <= table[i][1] + 10 * t0 - table[i][1], "start time : " + str(i+1)
    # xi + ti <= t_end_i + (1-ci)*(5*t0-t_end_i)
    portfolio += x_i[i] + c_i[i] * (10 * t0 - table[i][2]) <= table[i][2] + 10 * t0 - table[i][1] - table[i][0], "end time : " + str(i+1)
    # ci >= c_0_i
    portfolio += c_i[i] >= table[i][6], "must or not : " + str(i+1)

In [149]:
# добавляем условие на общий бюджет
# аффинная сумма ci и расходы * ti
budjet = LpAffineExpression([(c_i[i], table[i][0] * table[i][4]) for i in range(N)])
portfolio += budjet <= b0, "budjet"

In [150]:
# решаем систему линейных уравнений
portfolio.solve()

1

In [151]:
# выводим значения переменных
for v in portfolio.variables():
    print(v.name, "=", v.varValue)

do_or_not_1 = 1.0
do_or_not_2 = 1.0
do_or_not_3 = 1.0
do_or_not_4 = 0.0
do_or_not_5 = 0.0
proj_1 = 0.0
proj_2 = 1.0
proj_3 = 1.0
proj_4 = 39.0
proj_5 = 48.0


In [152]:
# выводим значение целевой функции
print("max = ", value(portfolio.objective))

max =  20.0


In [153]:
portfolio.writeLP("seva.lp")